Compare House TCJA total change in liability when using default weights vs. those provided by Ernie Tedeschi. 

Uses behavior file provided by Matt Jensen.

In [1]:
from taxcalc import *
from taxcalc.utils import *

In [2]:
recs = Records(data=os.path.join(Records.CUR_PATH, 'cps.csv.gz'),
               weights='cps_weights_alt.csv',
               adjust_ratios=None,
               start_year=Records.CPSCSV_YEAR)

In [15]:
r.close()

In [27]:
url

'https://raw.githubusercontent.com/open-source-economics/Tax-Calculator/master/taxcalc/reforms/TCJA_Reconciliation.json'

In [23]:
import urllib
url = REFORM_PATH + 'TCJA_Reconciliation.json'
jsonurl = urllib.urlopen(url)

In [26]:
json.loads(jsonurl.read())

ValueError: No JSON object could be decoded

In [35]:
import requests, urllib2

REFORM_PATH = 'https://raw.githubusercontent.com/open-source-economics/Tax-Calculator/master/taxcalc/reforms/'
url = REFORM_PATH + 'TCJA_Reconciliation.json'

r = requests.get(url)
reform_text = urllib2.urlopen(url).read()

In [38]:
reform = Calculator.read_json_param_objects(reform_text, #'TCJA_Reconciliation.json',
                                            'JCT_Behavior.json')
pol = Policy()
pol.implement_reform(reform['policy'])
con = Consumption()
con.update_consumption(reform['consumption'])
beh = Behavior()
beh.update_behavior(reform['behavior'])

In [39]:
def total_combined_liability(calc):
    # combined is combined tax liability while s006 is weight.
    return (calc.records.combined * calc.records.s006).sum()

In [40]:
def print_combined_liability(baseline_calc, reform_calc):
    # Calculate and print liabilities.
    baseline = total_combined_liability(baseline_calc)
    reformed = total_combined_liability(reform_calc)
    diff = reformed - baseline
    print ('Combined Liability - Baseline: {:0.2f}'.format(baseline))
    print ('Combined Liability - Reform: {:>18.2f}'.format(reformed))
    print ('-' * 47)
    print ('Difference: {:35.2f}'.format(diff))

In [41]:
calc = Calculator(records=recs, policy=Policy())
calc.advance_to_year(2018)
calc.calc_all()
calc_reform = Calculator(records=recs, policy=pol,
                         consumption=con, behavior=beh)
calc_reform.advance_to_year(2018)
calc_response = Behavior.response(calc, calc_reform)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [42]:
print_combined_liability(calc, calc_reform)

Combined Liability - Baseline: 2718451561189.63
Combined Liability - Reform:   2519669937647.26
-----------------------------------------------
Difference:                    -198781623542.36


In [43]:
print_combined_liability(calc, calc_response)

Combined Liability - Baseline: 2718451561189.63
Combined Liability - Reform:   2504231445940.50
-----------------------------------------------
Difference:                    -214220115249.13


In [45]:
calc_response.records

In [44]:
create_difference_table(calc.records, calc_response.records, groupby='weighted_deciles',
                        income_measure='expanded_income', tax_to_diff='combined')

AssertionError: 